<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/makingSmallCorpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-c970  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Dec 29 19:55 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '209206846'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
# doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls queries_train.json

# adding our python module to the cluster
sc.addFile("/home/dataproc/queries_train.json")
sys.path.insert(0,SparkFiles.getRootDirectory())



queries_train.json


Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
import json
with open("queries_train.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [ ]:
all_docs_id_from_jason = set()
for i in range(len(jsonObject)):
  for doc_id in list(jsonObject.items())[i][1]:
    print(doc_id)
    all_docs_id_from_jason.add(str(doc_id))

60283633
61073786
5676692
56289553
60774345
27306717
61592102
42163310
60952488
36439749
48530084
10589717
29129051
59892
612052
44254295
878659
54653881
51430647
66111204
22114132
55935213
41677925
17296107
61651800
9110929
67055
37497391
60744481
65464184
41974496
60616450
60463979
65967176
57069491
46208997
22144990
62372638
1074657
44240443
33463661
41974555
43603241
33700618
5027882
66423851
60754840
15474
1357127
636806
43033258
6271835
56480301
23133297
615418
73165
24470328
1833777
1380383
79449
4827661
387703
18863597
36827305
494299
194687
5591344
48490547
634139
42072639
44311171
29384326
1908019
296627
11263877
101942
2045465
56921904
128987
22888933
1072968
25490788
83449
884998
1151454
30640885
35072597
2535885
30861
51046955
13603
3060346
88380
19698110
72214
6236554
46105
1897206
10179411
25130414
5818361
1185840
20948989
48317971
509628
494528
11486091
50716473
24963841
296950
35804330
261193
15271
39000674
19988623
38156944
36794719
731640
14109784
10328235
25935906
1

In [ ]:
print(type(parquetFile))

# new_training_bucket = parquetFile.filter(lambda x: x['id'] in all_docs_id_from_jason)
new_training_bucket = parquetFile.filter("id in ({})".format(','.join(map(str, all_docs_id_from_jason))))

#write to a new bucket
# index_src = "new_training_bucket.pkl"
# index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
# new_training_bucket.write.parquet(index_dst)

<class 'pyspark.sql.dataframe.DataFrame'>


In [ ]:
new_training_bucket.show(10, truncate=True)

+-------+--------------------+--------------------+--------------------+
|     id|               title|                text|         anchor_text|
+-------+--------------------+--------------------+--------------------+
|4090453|         Money Train|'''''Money Train'...|[{23718565, Josep...|
|4093054|     Java annotation|In the Java compu...|[{15881, Java com...|
|4208015|         British Raj| 

The '''British...|[{4721, Imperial ...|
|4226137|     Hybrid tea rose|'''Hybrid tea''' ...|[{29640296, garde...|
|4253801|      Jumpman (logo)|The "'''Jumpman''...|[{1394509, Air Jo...|
|4294832|Java collections ...|thumb|java.util.C...|[{15881, Java}, {...|
|4335905|       Talulah Riley|'''Talulah Jane R...|[{22635683, St Tr...|
|4378282|Secondary ferment...|right|thumb|400px...|[{1455948, macera...|
|4387617|Grapefruit–drug i...|thumb|500px
Some ...|[{1099396, intera...|
|4416646|           Don Lapre|'''Donald D. Lapr...|[{19356538, Provi...|
+-------+--------------------+--------------------+

In [ ]:
# index_src = "new_training_bucket.pkl"
# index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
# !gsutil cp $index_src $index_dst

In [ ]:
# # if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
# %cd -q /home/dataproc
# !ls inverted_index_gcp.py

In [ ]:
# # adding our python module to the cluster
# sc.addFile("/home/dataproc/inverted_index_gcp.py")
# sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
# from inverted_index_gcp import InvertedIndex

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 
